# Standardni paketi

Pythonova knjižnica standardnih modulov in paket je precej obsežna (https://docs.python.org/3/library/). Ogledali si bomo zgolj par pogostejših.

## Čas in datum

Za delo s časom in datumi nam prideta prav modula `time` in `datetime`. Že prištevanje par dni k trenutnemu datumu je lahko kompleksna naloga, če vpeljemo še premik ure in časovne pasove, pa se to že cel projekt. Tu nam omenjena modula lahko precej olajšata delo.

### Time

Oglejmo si najprej modul `time` (https://docs.python.org/3/library/time.html). Funkcija `time.time()` (funkcija `time` iz modula `time`) nam vrne število sekund od nekega fiksnega predhodnega trenutka. Običajno je to 1. januar 1970. Ker število sekund od leta 1970 človeku ne pove prav dosti, si lahko s funkcijo `localtime` to vrednost pretvorimo v leto, mesec, dan, ure, minute in sekunde.

In [1]:
import time

now = time.time()
print(now)
print(time.localtime(0))
print(time.localtime(now))
print(time.localtime())

1637661470.6473315
time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=1, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)
time.struct_time(tm_year=2021, tm_mon=11, tm_mday=23, tm_hour=10, tm_min=57, tm_sec=50, tm_wday=1, tm_yday=327, tm_isdst=0)
time.struct_time(tm_year=2021, tm_mon=11, tm_mday=23, tm_hour=10, tm_min=57, tm_sec=50, tm_wday=1, tm_yday=327, tm_isdst=0)


Prav nam pride tudi za merjenje časa izvajanja programa. Ker opazujemo razliko med dvema časoma, je povsem nerelevantno, od katerega trenutka v zgodovini je ta čas merjen. Funkcija `time` vrne stenski čas, kot bi ga prebrali z ure na steni.

In [2]:
def f(n):
    return sum(i**2 for i in range(n))

start_time = time.time()
f(10**6)
end_time = time.time()
print("time        :", end_time - start_time)

time        : 0.22266936302185059


Boljšo natančnost lahko dosežemo z uporabo funkcije `perf_counter`. Smiselne so samo razlike med dvema tako izmerjenima časoma, saj je referenčna točka, od koder se meri, nedefinirana.

In [3]:
def f(n):
    return sum(i**2 for i in range(n))

start_time = time.time()
start_perf = time.perf_counter()
f(10**3)
end_time = time.time()
end_perf = time.perf_counter()
print("time        :", end_time - start_time)
print("perf_counter:", end_perf - start_perf)

time        : 0.0
perf_counter: 0.00023410000000012587


Včasih pa nas ne zanima sistemski čas, temveč zgolj čas, ki ga je porabil naš program. Ta lahko čaka na kakšno vhodno/izhodno enoto (npr. disk), ali pa ima na sistemu višjo prioriteto nek drug proces, zato naš program čaka oz. pride na vrsto občasno. Količino časa, ki jo porabi naš program, lahko dobimo s funkcijo `process_time`.

In [4]:
def f(n):
    time.sleep(1)
    sum(i**2 for i in range(n))

start_time = time.time()
start_perf = time.perf_counter()
start_proc = time.process_time()
f(10**6)
end_time = time.time()
end_perf = time.perf_counter()
end_proc = time.process_time()
print("time        :", end_time - start_time)
print("perf_counter:", end_perf - start_perf)
print("process_time:", end_proc - start_proc)

time        : 1.2276315689086914
perf_counter: 1.2318521000000011
process_time: 0.21875


Katero funkcijo bomo uporabili, je povsem odvisno od primera uporabe. Moramo pa biti pozorni, da pogosta uporaba funkcij za odčitavanje časa, prav tako zahteva nezanemarljivo količino časa.

In [5]:
from time import get_clock_info
for clock in ['time', 'perf_counter', 'process_time', 'monotonic']:
    print(f"{clock:13} {get_clock_info(clock)}")

time          namespace(adjustable=True, implementation='GetSystemTimeAsFileTime()', monotonic=False, resolution=0.015625)
perf_counter  namespace(adjustable=False, implementation='QueryPerformanceCounter()', monotonic=True, resolution=1e-07)
process_time  namespace(adjustable=False, implementation='GetProcessTimes()', monotonic=True, resolution=1e-07)
monotonic     namespace(adjustable=False, implementation='GetTickCount64()', monotonic=True, resolution=0.015625)


### Datetime

Modul `datetime` (https://docs.python.org/3/library/datetime.html) nam ponuja funkcije za delo z datumi in časom. Funkcija `datetime.now` nam vrne trenutni datum in čas. Enostavno lahko dostopamo do posameznih atributov (npr. leto), številne druge funkcije pa nam omogočajo tudi določanje dneva v tednu in podobne stvari.

In [6]:
from datetime import datetime

now = datetime.now()
print(now, repr(now))
print(now.year, now.month, now.day)
print(now.weekday())  # 0=ponedeljek, 1=torek, ...

2021-11-23 10:57:59.634172 datetime.datetime(2021, 11, 23, 10, 57, 59, 634172)
2021 11 23
1


Datetime objekt lahko sestavimo tudi s poljubnim datumom in časom. Lahko ga zgradimo neposredno, ali pa najprej ločeno sestavimo datum in čas, ter ju nato združimo.

In [7]:
from datetime import date, time

datum = date(2000, 11, 1)
cas = time.fromisoformat("08:45")
print([datum, cas])
datum_cas = datetime.combine(datum, cas)
print(datum_cas)

[datetime.date(2000, 11, 1), datetime.time(8, 45)]
2000-11-01 08:45:00


Pogosta naloga pri delu z datumi in časom je njihova predstavitev v obliki niza (`strftime` - format) ali branje iz takega niza (`strptime` - parse). Oznake za definiranje oblike so opisane v dokumentaciji (https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).

In [8]:
print(datum_cas.strftime("%d.%m.%Y (%H:%M)"))
print(datetime.strptime("01.11.2000 (08:30)", "%d.%m.%Y (%H:%M)"))

01.11.2000 (08:45)
2000-11-01 08:30:00


Rezultat odštevanja dveh datumov je posebnega tipa `timedelta`, ki hrani razliko v dnevih in sekundah. Podobno lahko ustvarimo objekt tipa `timedelta` in ga prištejemo trenutnemu datumu, s čimer dobimo nov primerno zamaknjen datum.

In [9]:
from datetime import timedelta

konec = datetime(datetime.now().year, 12, 31, 23, 59)
now = datetime.now()
cas = konec - now
print("Preostanek časa:", repr(cas), cas.days)

en_dan = timedelta(1)
print("Tri dni kasneje:", now + 3*en_dan)

Preostanek časa: datetime.timedelta(days=38, seconds=46858, microseconds=157836) 38
Tri dni kasneje: 2021-11-26 10:58:01.842164


Modul podpira tudi delo s časovnimi pasovi in premiki ure. To je posebne vrste komplikacija, ki se ji bomo na tem mestu izognili.

## Decimalna števila in ulomki

Python omogoča operacije s poljubno velikimi celimi števili. To pa ne drži za realna števila predstavljena s plavajočo vejico (`float`). Pri operacijah z njimi moramo namreč upoštevati računske napake, ki se prikradejo v izračune. Pri tem si lahko pomagamo z moduloma `decimal` in `fraction`.

### Računske napake

In [10]:
x = pow(10.0, 20)
print(x == x+1)

True


Ja, prav vidite. Števila imajo omejeno natačnost. Pri 64-bitnih številih je to približno 15 decimalnih mest. Zato se število 1 izgubi v primerjavi z večjim 21-mestnim številom.

In [11]:
x = sum(0.5 for i in range(10))
y = sum(0.1 for i in range(10))
print(x==5, y==1)

True False


Če seštejemo deset polovic, dobimo 5. Če seštejemo deset desetin, pa ne dobimo ena? Vrednost 0.5 ima lepo predstavitev v dvojiškem sistemu. Število 0.1 pa ima v dvojiškem neskončen zapis, zato se nekje zaokroži.

In [12]:
x = sum(1/i for i in range(1,100000))
y = sum(1/i for i in reversed(range(1,100000)))
print(x, y, x==y)

12.090136129863335 12.090136129863408 False


Še iz osnovne šole vemo, da je vsota komutativna in jo lahko seštevamo v poljubnem vrstnem redu. Žal se Python ne strinja (kakor tudi večina drugih programskih jezikov). Težava ponovno izhaja iz seštevanja števil različnih velikostnih razredov. V prvem primeru začnemo z velikimi števili in jim prištevamo vedno manjša, kar vodi do računskih napak. Drugi način je zato pravilnejši.

### Decimal

Razred `Decimal` iz modula `decimal` (https://docs.python.org/3/library/decimal.html) nam omogoča izvajanje operacij v desetiškem sistemu s poljubno natančnostjo. Poskusimo ponovno sešteti deset desetin, vendar tokrat z uporabo razreda `Decimal`. Previdni moramo biti, da ne ustvarimo objekta iz števila `0.1`, ki že vsebuje zaokrožitveno napako, temveč iz niza `"0.1"`

In [13]:
from decimal import Decimal

v = sum(Decimal("0.1") for i in range(10))
print(v, v == 1.0)

1.0 True


Nastavimo lahko poljubno natančnost pri računanju z objekti razreda `Decimal`. Do te nastavitve pridemo preko metode `getcontext()`. Ta nam vrne objekt, ki predstavlja trenutni kontekst računanja, kjer lahko spremenimo natančnost.

In [14]:
from decimal import getcontext

print(1/31)
print(Decimal(1) / Decimal(31))
getcontext().prec = 50
print(Decimal(1) / Decimal(31))

0.03225806451612903
0.03225806451612903225806451613
0.032258064516129032258064516129032258064516129032258


Pri zaokroževanju se polovice običajno ne zaokrožujejo navzgor, temveč k sodi vrednosti (`ROUND_HALF_EVEN`). Preko konteksta računanja lahko izberemo tudi drugačno obnašanje (https://docs.python.org/3/library/decimal.html#rounding-modes).

In [15]:
getcontext().prec = 1
print(Decimal("0.14") + Decimal("0.51"))  # 0.65 -> 0.6
print(Decimal("0.14") + Decimal("0.41"))  # 0.55 -> 0.6

from decimal import ROUND_UP
getcontext().rounding = ROUND_UP
print(Decimal("0.12") + Decimal("0.30"))  # 0.42 -> 0.5

0.6
0.6
0.5


### Fraction

Če želimo biti povsem natančni, lahko uporabimo razred `Fraction` iz modula `fractions` (https://docs.python.org/3/library/fractions.html), kjer pravzaprav računamo s celimi števili v števcih in imenovalcih. Glavna atributa sta `numerator` in `denominator`.

In [16]:
from fractions import Fraction

f = Fraction(1, 3)
print(f, f.numerator, f.denominator)

print(Fraction(1,3) + Fraction(3,4))

1/3 1 3
13/12


Tako kot pri razredu `Decimal`, moramo biti tudi tu pozorni, da ne vnesemo računske napake še pred začetkom računanja z natančnimi števili.

In [17]:
print(Fraction(0.2))
print(Fraction("0.2"))

3602879701896397/18014398509481984
1/5


## Formati podatkovnih datotek

Podatke shranjujemo v datoteke za arhiviranje, izmenjavo, shranjevanje vmesnih izračunov, ... Izbrani format je lahko tekstovna ali binarna datoteka. Prva je enostavno berljiva, druga pa tipično bolj učinkovita. Oglejmo si par najpogostejših in Pythonovo podporo zanje.

### Context Manager

Pri delu s podatkovnimi datotekami različnih formatov se boste pogosto srečali z `with` blokom in z njim povezanimi context managerji.

Kot že veste, je tipičen način dela z datotekami sledeč: odpremo datoteko, nekaj počnemo z njo in jo na koncu zapremo, da je na voljo drugim.

In [18]:
f = open("data.txt", "r")
print(f.readline())
f.close()

Ljubljana 286745 164 295 1000


Program lahko malo skrajšamo z uporabo `with` bloka, ki poskrbi za zapiranje datoteke po zaključku izvajanja bloka.

In [19]:
with open("data.txt", "r") as f:
    print(f.readline())

Ljubljana 286745 164 295 1000


With blok sprejme objekt (context manager) z implementiranima metodama `__enter__` in `__exit__`, ki se izvedeta pred začetkom in po koncu izvajanja bloka. S tem poskrbimo, da za zaprejo datoteke, prekinejo povezave z bazo ali kaj tretjega, tudi v primeru napak, do katerih lahko pride med izvajanjem bloka. Zato metoda `__exit__` sprejme kot argumente tudi tip, vrednost in traceback (seznam klicev funkcij) v primeru napake. Lasten context manager za branje datoteke, bi izgledal tako.

In [20]:
class DatotekaZaBranje:
    def __init__(self, ime):
        self.dat = open(ime, "r")
    def __enter__(self):
        return self.dat
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.dat.close()

with DatotekaZaBranje("data.txt") as f:
    print(f.readline())

Ljubljana 286745 164 295 1000


Pred nadaljevanjem si pripravimo še pomožno funkcijo, ki bo prebrala in vrnila vsebino datoteke. Prav nam bo prišla za ogled, s kakšno datoteko imamo opravka oz. kakšno datoteko smo ustvarili.

In [21]:
def read(file_name, *args, **kwargs):
    with open(file_name, "r", *args, **kwargs) as f:
        return f.read()

print(read("data.txt"))

Ljubljana 286745 164 295 1000


### CSV

V comma-separated values formatu lahko shranimo tabelarične podatke. Vsi vnosi so opisani z enakimi atributi, tako da jih lahko naštejemo vsakega v svoji vrstici s presledki ločenimi vrednostmi.

In [22]:
print(read("data.csv"))

ime,priimek,starost,naslov
Ana,Novak,22,Gosposvetska 12
Janez,Kuhar,20,Barjanska 1


Osnovni csv format je dovolj enostaven, da bi ga lahko brez večjih težav prebrali tudi brez posebnih knjižnic.

In [23]:
with open("data.csv", "r") as f:
    data = [line.strip().split(',') for line in f]
print(data)

[['ime', 'priimek', 'starost', 'naslov'], ['Ana', 'Novak', '22', 'Gosposvetska 12'], ['Janez', 'Kuhar', '20', 'Barjanska 1']]


Težava pa nastane, če se v podatkih nahajajo ločila, ki jih uporabljamo za ločevanje stolpcev. V tem primeru vsebino obdamo z narekovaji in s tem definiramo, kaj paše skupaj.

In [24]:
print(read("data2.csv"))

ime,priimek,starost,naslov
Ana,Novak,22,"Gosposvetska 12, 2000 Maribor"
Janez,Kuhar,20,"Barjanska 1, 1000 Ljubljana"


Branje takega formata je že precej bolj zapleteno, saj moramo ugotoviti, katera ločila za nahajajo znotraj in katera izven narekovajev. Na tem mestu pa si bomo pomagali z modulom `csv` (https://docs.python.org/3/library/csv.html). Z uporabo funkcije `csv.reader` lahko enostavno preberemo posamezne vrstice v sezname nizov.

In [25]:
import csv

with open("data2.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['ime', 'priimek', 'starost', 'naslov']
['Ana', 'Novak', '22', 'Gosposvetska 12, 2000 Maribor']
['Janez', 'Kuhar', '20', 'Barjanska 1, 1000 Ljubljana']


Pogost vir tabelaričnih podatkov so razpredelnice iz programov kot je npr. Excel. Če shranimo razpredelnico v csv, dobimo malenkost posebno obliko. Vnosi so ločeni s podpičji, datoteka pa je shranjena v UTF-8 formatu s posebno oznako na začetku datoteke (byte order mark), kar lahko preberemo z uporabo encodinga utf-8-sig (https://docs.python.org/3/library/codecs.html#encodings-and-unicode).

In [26]:
print(read("tecajnica_excel.csv", encoding="utf-8-sig"))

Država;Valuta;Nakupni tečaj;Prodajni tečaj
ZDA;USD;1,1913;1,1633
Velika Britanija;GBP;0,9045;0,8785
Švica;CHF;1,0965;1,0605
Hrvaška;HRK;7,664;7,454
Madžarska;HUF;359,65;350,85
Srbija;RSD;119,43;115,63
Bosna in Hercegovina;BAM;1,998;1,917
Republika Severna Makedonija;MKD;62,11;60,29
Rusija;RUB;93,54;88,14
Češka;CZK;26,94;25,94
Poljska;PLN;4,557;4,409
Bolgarija;BGN;1,979;1,925
Norveška;NOK;10,895;10,555
Švedska;SEK;10,346;10,026
Danska;DKK;7,5575;7,3335
Avstralija;AUD;1,6405;1,5965
Kanada;CAD;1,5635;1,5155
Japonska;JPY;125,8;122



Na tem primeru bomo ilustrirali uporabo razredov `DictReader` in `DictWriter`. Navaden csv reader pretvori vsak vnos v seznam, DictReader pa v slovar, v katerem ključi ustrezajo imenom atributov iz prve vrstice csv datoteke. Bralniku kot ločilo (`delimiter`) med vnosi podamo podpičje. Zaradi uporabe decimalne vejice v številskih vrednostih izvedemo še zamenjavo vejic s pikami in pretvorimo števila v float.

In [27]:
with open("tecajnica_excel.csv", "r", encoding="utf-8-sig") as f:
    reader = csv.DictReader(f, delimiter=';')
    data = []
    for row in reader:
        for key, value in row.items():
            if "tečaj" in key: row[key] = float(value.replace(",", "."))
        data.append(row)
print(data)

[{'Država': 'ZDA', 'Valuta': 'USD', 'Nakupni tečaj': 1.1913, 'Prodajni tečaj': 1.1633}, {'Država': 'Velika Britanija', 'Valuta': 'GBP', 'Nakupni tečaj': 0.9045, 'Prodajni tečaj': 0.8785}, {'Država': 'Švica', 'Valuta': 'CHF', 'Nakupni tečaj': 1.0965, 'Prodajni tečaj': 1.0605}, {'Država': 'Hrvaška', 'Valuta': 'HRK', 'Nakupni tečaj': 7.664, 'Prodajni tečaj': 7.454}, {'Država': 'Madžarska', 'Valuta': 'HUF', 'Nakupni tečaj': 359.65, 'Prodajni tečaj': 350.85}, {'Država': 'Srbija', 'Valuta': 'RSD', 'Nakupni tečaj': 119.43, 'Prodajni tečaj': 115.63}, {'Država': 'Bosna in Hercegovina', 'Valuta': 'BAM', 'Nakupni tečaj': 1.998, 'Prodajni tečaj': 1.917}, {'Država': 'Republika Severna Makedonija', 'Valuta': 'MKD', 'Nakupni tečaj': 62.11, 'Prodajni tečaj': 60.29}, {'Država': 'Rusija', 'Valuta': 'RUB', 'Nakupni tečaj': 93.54, 'Prodajni tečaj': 88.14}, {'Država': 'Češka', 'Valuta': 'CZK', 'Nakupni tečaj': 26.94, 'Prodajni tečaj': 25.94}, {'Država': 'Poljska', 'Valuta': 'PLN', 'Nakupni tečaj': 4.557, '

Take podatke lahko zapišemo nazaj v csv datoteko. Pri odpiranju datoteke moramo podati argument `newline=''`, da objekt, ki predstavlja datoteko, ne izvaja kakšnih samoiniciativnih čarovnij z novimi vrsticami, ampak to prepusti csv modulu. Najprej izpišemo prvo vrstico z imeni atributov, ki smo jih dobili iz ključev slovarja, nato pa še preostanek podatkov.

In [28]:
with open("tecajnica.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys(), quoting=csv.QUOTE_NONNUMERIC)
    writer.writeheader()
    writer.writerows(data)

Za konec preverimo, ali vsebina datoteka ustreza pričakovanjem.

In [29]:
print(read("tecajnica.csv", encoding="utf-8"))

"Država","Valuta","Nakupni tečaj","Prodajni tečaj"
"ZDA","USD",1.1913,1.1633
"Velika Britanija","GBP",0.9045,0.8785
"Švica","CHF",1.0965,1.0605
"Hrvaška","HRK",7.664,7.454
"Madžarska","HUF",359.65,350.85
"Srbija","RSD",119.43,115.63
"Bosna in Hercegovina","BAM",1.998,1.917
"Republika Severna Makedonija","MKD",62.11,60.29
"Rusija","RUB",93.54,88.14
"Češka","CZK",26.94,25.94
"Poljska","PLN",4.557,4.409
"Bolgarija","BGN",1.979,1.925
"Norveška","NOK",10.895,10.555
"Švedska","SEK",10.346,10.026
"Danska","DKK",7.5575,7.3335
"Avstralija","AUD",1.6405,1.5965
"Kanada","CAD",1.5635,1.5155
"Japonska","JPY",125.8,122.0



### JSON

Kadar podatki niso zgolj tabelarični, temveč imajo večnivojsko (gnezdeno) strukturo, nam pride prav format JSON (https://docs.python.org/3/library/json.html).

In [30]:
data = [{"ime": "Janez", "ocene": [("P1", 6), ("P2", None)]},
        {"ime": "Teja", "ocene": [("P1", 10), ("P2", 7)]}]

import json

with open("ocene.json", "w") as f:
    json.dump(data, f, indent=4)

print(read("ocene.json"))

[
    {
        "ime": "Janez",
        "ocene": [
            [
                "P1",
                6
            ],
            [
                "P2",
                null
            ]
        ]
    },
    {
        "ime": "Teja",
        "ocene": [
            [
                "P1",
                10
            ],
            [
                "P2",
                7
            ]
        ]
    }
]


Gre za tekstovno datoteko, ki ima za človeka enostavno berljiv format. Vanj lahko med drugim shranimo nize, števila, sezname, slovarje (https://docs.python.org/3/library/json.html#py-to-json-table). Terke in seznami so v json datoteki shranjeni v enaki obliki, zato pri branju vedno dobimo sezname.

In [31]:
with open("ocene.json", "r") as f:
    data = json.load(f)
print(data)

[{'ime': 'Janez', 'ocene': [['P1', 6], ['P2', None]]}, {'ime': 'Teja', 'ocene': [['P1', 10], ['P2', 7]]}]


### Pickle

Če podatki ne ustrezajo tipom, ki jih zahteva JSON, temveč gre za neke poljubne objekte v Pythonu, si lahko pomagamo s formatom pickle (https://docs.python.org/3/library/pickle.html). Ta je namenjen prav serializaciji (pretvarjanju v niz/zaporedje) Pythonovih objektov. Rezultat je binarna datoteka, zato moramo pri odpiranju datoteke dodati zastavico `b`, da se zaradi avtomatskih sprememb (npr. oznak za konec vrstice) ne spremeni vsebina. V isto datoteko lahko shranimo tudi več objektov.

In [32]:
import pickle
from datetime import date
from fractions import Fraction

with open("data.pickle", "wb") as f:
    pickle.dump(date.today(), f)
    pickle.dump(Fraction(1, 3), f)

Poglejmo vsebino datoteke.

In [33]:
print(read("data.pickle"))

€•        Œdatetime”Œdate”“”Cå”…”R”.€•$       Œ	fractions”ŒFraction”“”Œ1/3”…”R”.


Datoteke ne moremo obravnavati kot navadno tekstovno datoteko, čeprav so določeni deli shranjeni v obliki teksta. Pri nalaganju vsebine iz pickle datoteke dobimo Pythonove objekte, s katerimi lahko izvajamo operacije kot pred shranjevanjem.

In [34]:
with open("data.pickle", "rb") as f:
    print(pickle.load(f))
    print(pickle.load(f) + 2)

2021-11-23
7/3


Pickle pa ima tudi svoje nevarnosti. Lastnim razredom, lahko preko metode `__reduce__` definiramo način, kako naj se shranijo. Ali pa namesto tega naredimo kakšno drugo grdobijo. Spodnji primer pri shranjevanju in nalaganju izvede še dodaten sistemski ukaz.

In [35]:
import os
class Oseba:
    def __init__(self, ime):
        self.ime = ime
    @classmethod
    def _reconstruct(cls, ime):
        print("operacijski sistem:", os.getenv("OS"))
        return cls(ime)
    def __reduce__(self):
        print("delovna mapa:", os.getcwd())
        return Oseba._reconstruct, (self.ime,)

with open("dir.pickle", "wb") as f:
    pickle.dump(Oseba("Ana"), f)

delovna mapa: C:\Tomaz\FRI\PEF - Programiranje 2\PEF-P2


Če nič hudega sluteči naložimo pickle datoteko, v katero je nekdo shranil objekt s zlonamerno kodo, si brez težav pobrišemo celoten disk ali kaj podobno neugodnega. Ker lahko taka datoteka vsebuje poljubno kodo, nalagamo objekte samo iz virov, ki jim zaupamo.

In [36]:
with open("dir.pickle", "rb") as f:
    o = pickle.load(f)
print(o.ime)

operacijski sistem: Windows_NT
Ana
